## A python stream generating application

In [1]:
import time
import csv
from socket import *

### Here we should implement mainly two functions:
1. A method that can take 5 records and 1 record from both dataset per second
2. Sent the retrieved data as text to 

In [2]:
with open('./ClimateData-Part2.csv','r') as c2, open ('./FireData-Part2.csv','r') as f2:
    climate = list(csv.reader(c2))[1:]
    fire = list(csv.reader(f2))[1:]
    

In [3]:
def generate_streams():
    pass

In [4]:
len(climate)

366

In [5]:
len(fire)//5 +1

534

In [ ]:
def generator():
    for i in range(533):
        try:
            yield fire[5*i:5*(i+1)], climate[i:i+1]
        except:
            yield fire[5*i:5*(i+1)]
a = generator()

#### another question is that, everything send through socket is byte, is it possible that we reverse it back to list and retain its original information?
- The **Answer** is json!

In [ ]:
import json

In [ ]:
# list to json str
t = json.dumps(next(a))

In [ ]:
# json str to byte and back to json str
t = t.encode()
t.decode()

In [ ]:
# load json str back to list
json.loads(t)[0]

In [ ]:
import socket

In [ ]:
import socket

def netcat(host, port, content):
    s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    s.connect((host, int(port)))
    s.sendall(content.encode())
    s.shutdown(socket.SHUT_WR)
    while True:
        data = s.recv(4096)
        if not data:
            break
        print(repr(data))
    s.close()

In [ ]:
netcat('localhost',9999,'aa aa aa aaaa')

We need to write both a server and client? 

In [ ]:
import socket
import time

host = 'localhost'
port = 9999

i = 0

s = socket.socket(socket.AF_INET, socket.SOCK_STREAM)

s.bind((host, port))
s.listen(1)

try:
    while True:
        conn, addr = s.accept()
        try:
            for j in range(10):
                conn.send(bytes("{}\n".format(i), "utf-8"))
                i += 1
                time.sleep(1)
            conn.close()
        except socket.error: pass
finally:
    s.close()

